# Interpreting Identity

# Setup

In [1]:
TRAIN_MODEL = True

In [2]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
import os

DEVELOPMENT_MODE = True
# Upgrade pip
%pip install --upgrade pip

from IPython import get_ipython

ipython = get_ipython()
# Code to automatically update the HookedTransformer code as its edited without restarting the kernel
ipython.magic("load_ext autoreload")
ipython.magic("autoreload 2")
ipython.run_line_magic("pip", "install ipympl")
ipython.run_line_magic("pip", "install scipy")
ipython.run_line_magic("pip", "install manim")
ipython.run_line_magic("pip", "install torch")
ipython.run_line_magic("pip", "install numpy<2")
ipython.run_line_magic("pip", "install einops")
ipython.run_line_magic("pip", "install transformer_lens")

Note: you may need to restart the kernel to use updated packages.


/tmp/ipykernel_219706/80073063.py:12: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_219706/80073063.py:13: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
/bin/bash: Zeile 1: 2: Datei oder Verzeichnis nicht gefunden
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import stuff
import torch
import numpy as np
import os
import tqdm.auto as tqdm
from pathlib import Path

import copy

from transformer_lens import HookedTransformer, HookedTransformerConfig, ActivationCache

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
# Define the location to save the model, using a relative path
PTH_LOCATION = "workspace/_scratch/identity.pth"

# Create the directory if it does not exist
os.makedirs(Path(PTH_LOCATION).parent, exist_ok=True)

cuda


# Model Training

## Config

In [4]:
INPUT_DIM = 3
OUTPUT_DIM = 3
frac_train = 1

# Optimizer config
lr = 1e-3
wd = 1e-2
betas = (0.9, 0.999)

num_epochs = 10000
checkpoint_every = 500

DATA_SEED = 599

## Define Task
* Define random function
* Define the dataset & labels

Input format:
|a|b|

In [5]:
def get_training_data(input_dim = INPUT_DIM, output_dim = OUTPUT_DIM, data_seed = DATA_SEED):
    torch.manual_seed(data_seed)
    a_vector = torch.arange(input_dim)
    dataset = torch.cartesian_prod(a_vector, a_vector).to(device)

    labels = torch.randint(0, output_dim, (dataset.shape[0],), device=device)
    train_data = dataset
    train_labels = labels
    # For now no test data
    test_data = dataset[0:0]
    test_labels = labels[0:0]
    print(train_data)
    print(train_labels)
    print(train_data.shape)
    print(test_data[:5])
    print(test_labels[:5])
    print(test_data.shape)
    return train_data, train_labels, test_data, test_labels

train_data, train_labels, test_data, test_labels = get_training_data()

tensor([[0, 0],
        [0, 1],
        [0, 2],
        [1, 0],
        [1, 1],
        [1, 2],
        [2, 0],
        [2, 1],
        [2, 2]], device='cuda:0')
tensor([2, 2, 1, 0, 2, 2, 0, 0, 2], device='cuda:0')
torch.Size([9, 2])
tensor([], device='cuda:0', size=(0, 2), dtype=torch.int64)
tensor([], device='cuda:0', dtype=torch.int64)
torch.Size([0, 2])


## Define Model

In [6]:

def get_seeded_model(seed = 999, input_dim = INPUT_DIM, output_dim = OUTPUT_DIM):
    cfg = HookedTransformerConfig(
        n_layers = 1,
        n_heads = 1,
        d_model = 2,
        d_head = 2,
        d_mlp = None,
        attn_only=True,
        act_fn = "relu",
        normalization_type=None,
        d_vocab=input_dim,
        d_vocab_out=output_dim,
        n_ctx=2,
        init_weights=True,
        device=device,
        seed = seed,
    )
    model = HookedTransformer(cfg)
    # Biases are enabled by default
    # for name, param in model.named_parameters():
    #     if "b_" in name:
    #         param.requires_grad = False
    return model

model = get_seeded_model(seed = 993)

## Define Optimizer + Loss

In [7]:
print(train_data)
print(model(train_data))

tensor([[0, 0],
        [0, 1],
        [0, 2],
        [1, 0],
        [1, 1],
        [1, 2],
        [2, 0],
        [2, 1],
        [2, 2]], device='cuda:0')
tensor([[[-0.0232, -0.3831,  0.0734],
         [ 0.3973,  0.0077,  0.2572]],

        [[-0.0232, -0.3831,  0.0734],
         [ 0.1674, -0.1676,  0.1479]],

        [[-0.0232, -0.3831,  0.0734],
         [ 0.1926,  0.8592, -0.0728]],

        [[-0.3110, -0.5506, -0.0756],
         [ 0.3419,  0.0151,  0.2194]],

        [[-0.3110, -0.5506, -0.0756],
         [ 0.1117, -0.1603,  0.1098]],

        [[-0.3110, -0.5506, -0.0756],
         [ 0.1047,  0.8789, -0.1347]],

        [[-0.0824,  0.2363, -0.1083],
         [ 0.4788, -0.1707,  0.3516]],

        [[-0.0824,  0.2363, -0.1083],
         [ 0.2537, -0.3527,  0.2469]],

        [[-0.0824,  0.2363, -0.1083],
         [ 0.3217,  0.6311,  0.0640]]], device='cuda:0',
       grad_fn=<ViewBackward0>)


In [8]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd, betas=betas)
def loss_fn(logits, labels):
    if len(logits.shape)==3:
        logits = logits[:, -1]
    logits = logits.to(torch.float64)
    log_probs = logits.log_softmax(dim=-1)
    correct_log_probs = log_probs.gather(dim=-1, index=labels[:, None])[:, 0]
    return -correct_log_probs.mean()
train_logits = model(train_data)
train_loss = loss_fn(train_logits, train_labels)
print(train_loss)
test_logits = model(test_data)
test_loss = loss_fn(test_logits, test_labels)
print(test_loss)
print("Uniform loss:")
print(np.log(INPUT_DIM * INPUT_DIM))

tensor(1.0589, device='cuda:0', dtype=torch.float64, grad_fn=<NegBackward0>)
tensor(nan, device='cuda:0', dtype=torch.float64, grad_fn=<NegBackward0>)
Uniform loss:
2.1972245773362196


## Actually Train

**Weird Decision:** Training the model with full batch training rather than stochastic gradient descent. We do this so to make training smoother and reduce the number of slingshots.

In [9]:
def print_stats(model):
    # Extract the p 2-dimensional tensors, vector i is vec[:, i]
    vec = model.W_U.data

    # Function to compute the angle between two vectors
    def compute_angle(v1, v2):
        cos_theta = torch.dot(v1, v2) / (torch.norm(v1) * torch.norm(v2))
        angle = torch.acos(cos_theta) * (180.0 / np.pi)
        return angle

    # Compute pairwise angles
    # for i in range(vec.shape[1]):
    #     for j in range(i+1, vec.shape[1]):
    #         angle = compute_angle(vec[:, i], vec[:, j])
    #         print(f"Angle between {i} and {j}: {angle.item():.2f}°")
    #     print(f"Norm of vector {i}: {torch.norm(vec[:, i]):.2f}")

print_stats(model)

In [10]:
def train_model(model, train_data, train_labels, test_data, test_labels, num_epochs = num_epochs, loss_target = None):
    optimizer = torch.optim.AdamW(
        model.parameters(), lr=lr, weight_decay=wd, betas=betas
    )
    train_losses = []
    test_losses = []
    model_checkpoints = []
    checkpoint_epochs = []
    if TRAIN_MODEL:
        for epoch in tqdm.tqdm(range(num_epochs)):
            train_logits = model(train_data)
            train_loss = loss_fn(train_logits, train_labels)
            train_loss.backward()
            train_losses.append(train_loss.item())
            if loss_target is not None and train_loss.item() < loss_target:
                print(f"Loss target {loss_target} reached with loss {train_loss.item()} at epoch {epoch}")
                break

            optimizer.step()
            optimizer.zero_grad()

            with torch.inference_mode():
                test_logits = model(test_data)
                test_loss = loss_fn(test_logits, test_labels)
                test_losses.append(test_loss.item())

            if ((epoch+1)%checkpoint_every)==0:
                checkpoint_epochs.append(epoch)
                model_checkpoints.append(copy.deepcopy(model.state_dict()))
                print_stats(model)
                print(f"Epoch {epoch} Train Loss {train_loss.item()} Test Loss {test_loss.item()}")
    if TRAIN_MODEL:
        torch.save(
            {
                "model":model.state_dict(),
                "config": model.cfg,
                "checkpoints": model_checkpoints,
                "checkpoint_epochs": checkpoint_epochs,
                "test_losses": test_losses,
                "train_losses": train_losses,
            },
            PTH_LOCATION)

train_model(
    model, train_data, train_labels, test_data, test_labels
)

In [11]:
if not TRAIN_MODEL:
    cached_data = torch.load(PTH_LOCATION)
    model.load_state_dict(cached_data['model'])
    model_checkpoints = cached_data["checkpoints"]
    checkpoint_epochs = cached_data["checkpoint_epochs"]
    test_losses = cached_data['test_losses']
    train_losses = cached_data['train_losses']

/tmp/ipykernel_219706/999483974.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cached_data = torch.load(PTH_LOCATION)


# Activations

In [12]:
def create_cache(model):
    input = train_data
    print("Transposed Input:\n", input.transpose(0, 1))
    logits, cache = model.run_with_cache(input)
    print("Labels: ", train_labels)
    print("Logits of last token:\n", logits[:, -1, :])
    print("Unembed:\n", model.W_U.data)
    print("Last layer before unembed:\n", cache.cache_dict["blocks.0.hook_resid_post"][:, -1, :])
    return cache

cache = create_cache(model)

Transposed Input:
 tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2],
        [0, 1, 2, 0, 1, 2, 0, 1, 2]], device='cuda:0')
Labels:  tensor([2, 2, 1, 0, 2, 2, 0, 0, 2], device='cuda:0')
Logits of last token:
 tensor([[ -6.4230,  -1.9307,  10.7630],
        [ -4.4916,  -8.5375,  12.1585],
        [ -1.7896,   9.3899,  -3.5204],
        [  6.9543,  -4.1992,  -7.7939],
        [ -4.3770,  -8.4497,  11.9299],
        [ -1.8799, -12.0370,  10.5191],
        [ 10.5641, -12.7535,  -7.6477],
        [  7.4779,  -9.8636,  -4.9022],
        [ -2.8386, -14.7257,  13.6992]], device='cuda:0',
       grad_fn=<SliceBackward0>)
Unembed:
 tensor([[-1.0812,  2.0407,  0.2946],
        [ 0.7589,  0.2098, -1.2724]], device='cuda:0')
Last layer before unembed:
 tensor([[ -0.0272,  -8.3789],
        [ -3.0794, -10.1822],
        [  4.2643,   3.8404],
        [ -2.5773,   5.6153],
        [ -3.0554,  -9.9970],
        [ -4.8838,  -9.3115],
        [ -6.6602,   4.5551],
        [ -5.0603,   2.7677],
        [ -5.9198, -12.

In [13]:
# Get a list of all activations stored in the cache, especially their names
cache.cache_dict

{'hook_embed': tensor([[[-1.1449,  0.6601],
          [-1.1449,  0.6601]],
 
         [[-1.1449,  0.6601],
          [ 0.3006, -0.1175]],
 
         [[-1.1449,  0.6601],
          [ 1.5386, -0.9611]],
 
         [[ 0.3006, -0.1175],
          [-1.1449,  0.6601]],
 
         [[ 0.3006, -0.1175],
          [ 0.3006, -0.1175]],
 
         [[ 0.3006, -0.1175],
          [ 1.5386, -0.9611]],
 
         [[ 1.5386, -0.9611],
          [-1.1449,  0.6601]],
 
         [[ 1.5386, -0.9611],
          [ 0.3006, -0.1175]],
 
         [[ 1.5386, -0.9611],
          [ 1.5386, -0.9611]]], device='cuda:0'),
 'hook_pos_embed': tensor([[[ 0.5616,  0.9027],
          [-0.6252, -1.5300]],
 
         [[ 0.5616,  0.9027],
          [-0.6252, -1.5300]],
 
         [[ 0.5616,  0.9027],
          [-0.6252, -1.5300]],
 
         [[ 0.5616,  0.9027],
          [-0.6252, -1.5300]],
 
         [[ 0.5616,  0.9027],
          [-0.6252, -1.5300]],
 
         [[ 0.5616,  0.9027],
          [-0.6252, -1.5300]],
 
      

# Animations

In [14]:
from manim import *

config.media_width = "80%"
config.verbosity = "WARNING"
config.preview = False

In [15]:
class VectorParams:
    def __init__(self, values = [], color = WHITE, label = ""):
        self.values = values
        self.color = color
        self.label = label
    def __repr__(self) -> str:
        return str(self.values) + "(" + str(self.color) + ")"

class Data:
    def __init__(self):
        self.vectors: list[list[VectorParams]] = [[]]
        self.steps = 0
        self.current_labels = set()

    def add_vector(self, vector, color = WHITE, label = ""):
        if label not in self.current_labels:
            self.current_labels.add(label)
            self.vectors[self.steps].append(VectorParams(values = vector, color = color, label = label))

    def next_step(self):
        self.steps += 1
        self.vectors.append([])
        self.current_labels = set()

    def add_vectors_at_hook(self, c: ActivationCache, hook: str, color0 = WHITE, color1 = WHITE, input_labels = None, input_colors = None):
        if input_labels is None:
            input_labels = ["" for i in range(c.cache_dict[hook].shape[0])]
        for i in range(c.cache_dict[hook].shape[0]):
            self.add_vector(c.cache_dict[hook][i][0].cpu(), color = color0, label = input_labels[i][:1])
            self.add_vector(
                c.cache_dict[hook][i][1].cpu(), color=color1 if input_colors is None else input_colors[i], label=input_labels[i][:2]
            )


def compile_data_vectors(cache, input_labels=None, input_colors=None):
    # Set default value as list of empty strings
    vectors = Data()
    vectors.add_vectors_at_hook(cache, "hook_embed", color1 = GRAY, input_labels=input_labels, input_colors=input_colors)
    vectors.next_step()
    vectors.add_vectors_at_hook(cache, "blocks.0.hook_resid_pre", input_labels=input_labels, input_colors=input_colors)
    vectors.next_step()
    vectors.add_vectors_at_hook(cache, "blocks.0.hook_resid_post", color0 = GRAY, input_labels=input_labels, input_colors=input_colors)
    # vectors.add_vectors_at_hook(cache, "blocks.0.hook_resid_mid")

    print(vectors.vectors)
    return vectors


vectors = compile_data_vectors(cache)

[[tensor([-1.1449,  0.6601])(#FFFFFF)], [tensor([-0.5833,  1.5629])(#FFFFFF)], [tensor([3.0017, 8.2604])(#888888)]]


In [16]:
def change_font_size(labeled_arrow: LabeledArrow, new_size):
    # print(labeled_arrow, labeled_arrow.submobjects)
    # print(labeled_arrow.submobjects[-1].font_size)
    if not isinstance(labeled_arrow, LabeledArrow):
        return
    label = labeled_arrow.submobjects[-1]
    box = labeled_arrow.submobjects[-2]
    if not isinstance(box, BackgroundRectangle):
        box = labeled_arrow.submobjects[-3]
    coords = label.get_center()
    # print(new_size)
    labeled_arrow.submobjects[-1] = MathTex(
        label.get_tex_string(), color=label.color, font_size=new_size
    )
    # print("size=", labeled_arrow.submobjects[-1].font_size)
    label = labeled_arrow.submobjects[-1]
    label.move_to(coords)
    box.width = label.width + 2 * box.buff
    box.height = label.height + 2 * box.buff

In [17]:
DOT_SCALE = 0.05
class VisualizeTransformer(MovingCameraScene):
    def construct(self):
        print("v=", vectors.vectors)
        axes = Axes(
            x_range = [-20, 20, 1],
            y_range = [-20, 20, 1],
            x_axis_config={
                "numbers_to_include": np.arange(-18, 18.1, 3),
                "font_size": 24
            },
            y_axis_config={
                "numbers_to_include": np.arange(-18, 18.1, 3), 
                "font_size": 24            
            },
            x_length = 40,
            y_length = 40,
            axis_config={"color": GREEN}
        )

        scale = ValueTracker(2)

        dots = VGroup()
        def update_scale(self):
            return
            # TODO: Make the scaling nicer
            self.stroke_width = 6 * scale.get_value()
            change_font_size(self, 48 * scale.get_value())
            # print("New font size: ", self.font_size)

        # Embedding arrows
        for i, t in enumerate(vectors.vectors[0]):
            # print(t, t.numpy())
            # arrow = LabeledArrow(
            #     start=ORIGIN,
            #     end=np.append(t.values.numpy(), 0),
            #     buff = 0,
            #     label = t.label,
            #     label_frame = False,
            #     label_color=YELLOW,
            #     color = t.color,
            #     max_stroke_width_to_length_ratio = 100,
            # )

            # arrow.add_updater(update_scale)
            # arrows.add(arrow)
            dot = LabeledDot(
                point=np.append(t.values.numpy(), 0),
                label=t.label,
                color=t.color,
                radius=DOT_SCALE * (len(vectors.vectors[0]) - i) + 0.2,
            )

            dot.add_updater(update_scale)
            dots.add(dot)

        # Transitioning the arrows through the model
        self.add(axes, axes.get_axis_labels(), dots)
        for step in range(1, len(vectors.vectors)):
            new_dots = VGroup()
            transition_arrows = VGroup()
            for i, t in enumerate(vectors.vectors[step]):
                # print(t, t.numpy())
                # new_arrow = LabeledArrow(
                #     start=ORIGIN,
                #     end=np.append(t.values.numpy(), 0),
                #     buff=0,
                #     label=t.label,
                #     label_frame=False,
                #     label_color=YELLOW,
                #     color=t.color,
                #     max_stroke_width_to_length_ratio=100,
                # )
                # new_arrow.add_updater(update_scale)
                # new_arrows.add(new_arrow)
                new_dot = LabeledDot(
                    point=np.append(t.values.numpy(), 0),
                    label=t.label,
                    color=t.color,
                    radius = DOT_SCALE * (len(vectors.vectors[step]) - i) + 0.2,
                )
                new_dot.add_updater(update_scale)
                new_dots.add(new_dot)

                transition_arrow = Arrow(
                    start=dots[i].arc_center,
                    end=new_dots[i].arc_center,
                    buff=0,
                    color=RED,
                )
                transition_arrow.add_updater(update_scale)
                transition_arrows.add(transition_arrow)

            view = SurroundingRectangle(new_dots)
            factor = max(
                view.width / self.camera.frame_width,
                view.height / self.camera.frame_height,
            )
            print(
                factor,
                self.camera.frame_width, view.width,
                self.camera.frame_height, view.height,
            )
            self.wait()
            self.play(FadeIn(transition_arrows), self.camera.auto_zoom(view, margin = 2), scale.animate.set_value(scale.get_value() * factor))
            self.wait()
            self.play(
                ReplacementTransform(dots, new_dots)
            )
            self.wait()
            self.play(FadeOut(transition_arrows))
            self.wait()
            dots = new_dots

        # Unembedding Arrows
        embedding_arrows = VGroup()
        data = model.W_U.data
        print("unembed: ", data)
        for i in range(model.W_U.data.size()[1]):
            embedding_arrow = LabeledArrow(
                start=ORIGIN,
                end=[data[0, i].item(), data[1, i].item(), 0],
                label=str(i),
                color=LIGHT_PINK,
                buff=0,
                max_stroke_width_to_length_ratio=100,
            )
            embedding_arrows.add(embedding_arrow)
        self.play(FadeIn(embedding_arrows))
        self.wait()

# v = VisualizeTransformer()
# v.construct()

In [18]:
%env CUDA_LAUNCH_BLOCKING=1
%env TORCH_USE_CUDA_DSA=1

env: CUDA_LAUNCH_BLOCKING=1
env: TORCH_USE_CUDA_DSA=1


In [19]:
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [20]:
%%manim -qh Video

import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'


input_dim = 4
output_dim = 2
train_data, train_labels, test_data, test_labels = get_training_data(input_dim, output_dim, data_seed=999)
# model = get_seeded_model(998, input_dim, output_dim)
# train_model(model, train_data, train_labels, test_data, test_labels, num_epochs = 10000, loss_target = 1/(input_dim ** 2 * 4))
cache = create_cache(model)
arrow_labels = ["".join([str(d.item()) for d in v]) for v in train_data]
colors = [BLUE, YELLOW, GREEN, RED]
arrow_colors = [colors[l] for l in train_labels]
vectors = compile_data_vectors(cache, input_labels=arrow_labels, input_colors = arrow_colors)
print("Labels: ", arrow_labels)

class Video(VisualizeTransformer):
    def construct(self):
        VisualizeTransformer.construct(self)

tensor([[0, 0],
        [0, 1],
        [0, 2],
        [0, 3],
        [1, 0],
        [1, 1],
        [1, 2],
        [1, 3],
        [2, 0],
        [2, 1],
        [2, 2],
        [2, 3],
        [3, 0],
        [3, 1],
        [3, 2],
        [3, 3]], device='cuda:0')
tensor([0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1], device='cuda:0')
torch.Size([16, 2])
tensor([], device='cuda:0', size=(0, 2), dtype=torch.int64)
tensor([], device='cuda:0', dtype=torch.int64)
torch.Size([0, 2])
Transposed Input:
 tensor([[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
        [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]], device='cuda:0')


../aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [0,0,0], thread: [14,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [0,0,0], thread: [15,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [0,0,0], thread: [30,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [0,0,0], thread: [31,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [0,0,0], thread: [46,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [0,0,0], thread: [47,0,0] Assertion `-sizes[i] <

RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling cublasLtMatmul with transpose_mat1 0 transpose_mat2 0 m 2 n 32 k 2 mat1_ld 2 mat2_ld 2 result_ld 2 abcType 0 computeType 68 scaleType 0